<a href="https://colab.research.google.com/github/pinedance/gym-AI-NLP/blob/main/%ED%95%9C%EB%AC%B8_%EC%9E%90%EB%8F%99_%ED%91%9C%EC%A0%90_%ED%85%8C%EC%8A%A4%ED%8A%B8_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고문헌 자동 표점 테스트

source
* [raynardj/classical-chinese-punctuation-guwen-biaodian](https://huggingface.co/raynardj/classical-chinese-punctuation-guwen-biaodian)

REF
* [BertForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertForTokenClassification)

In [ ]:
%%bash
# pip install transformers

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [ ]:
model_name = "raynardj/classical-chinese-punctuation-guwen-biaodian"
tokenizer = AutoTokenizer.from_pretrained( model_name )
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
example_input = "郡邑置夫子庙于学以嵗时释奠盖自唐贞观以来未之或改我宋有天下因其制而损益之姑苏当浙右要区规模尤大更建炎戎马荡然无遗虽修学宫于荆榛瓦砾之余独殿宇未遑议也每春秋展礼于斋庐已则置不问殆为阙典今寳文阁直学士括苍梁公来牧之明年实绍兴十有一禩也二月上丁修祀既毕乃愓然自咎揖诸生而告之曰天子不以汝嘉为不肖俾再守兹土顾治民事神皆守之职惟是夫子之祀教化所基尤宜严且谨而拜跪荐祭之地卑陋乃尔其何以掲防妥灵汝嘉不敢避其责曩常去此弥年若有所负尚安得以罢輭自恕复累后人乎他日或克就绪愿与诸君落之于是谋之僚吏搜故府得遗材千枚取赢资以给其费鸠工庀役各举其任嵗月讫工民不与知像设礼器百用具修至于堂室廊序门牖垣墙皆一新之"


In [ ]:
example_input = "" # @param {type:"string"}

In [ ]:
inputs = tokenizer( example_input, add_special_tokens=False, return_tensors="pt" )

In [ ]:
with torch.no_grad():
    logits = model( **inputs ).logits

In [ ]:
predicted_token_class_ids = logits.argmax(-1)

In [ ]:
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

In [ ]:
list( set( predicted_tokens_classes ) )

In [ ]:
labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

0.09

In [ ]:
def build_output( input_text, predicted_class ):
    if len(input_text) != len(predicted_class):
        print( "입력 텍스트와 결과 텍트트의 길이가 서로 다릅니다. ")
        return None
    output_list = list()
    for a, b in zip( input_text, predicted_class ):
        output_list.append( a )
        if b != "O":
            output_list.append( b )
    rst = "".join( output_list )
    rst = rst.replace("", "")
    return rst

In [ ]:
build_output( example_input, predicted_tokens_classes )

'郡邑，置夫子庙于学，以嵗时释奠。盖自唐。贞观以来，未之或改。我宋有天下因其制而损益之。姑苏当浙右要区，规模尤大，更建炎戎马，荡然无遗。虽修学宫于荆榛瓦砾之余，独殿宇未遑议也。每春秋展礼于斋庐，已则置不问，殆为阙典。今寳文阁直学士括苍梁公来牧之。明年，实绍兴十有一禩也。二月，上丁修祀既毕，乃愓然自咎，揖诸生而告之曰"天子不以汝嘉为不肖，俾再守兹土，顾治民事神，皆守之职。惟是夫子之祀，教化所基，尤宜严且谨。而拜跪荐祭之地，卑陋乃尔！其何以掲防妥灵？汝嘉不敢避其责。曩常去此弥年，若有所负，尚安得以罢輭自恕，复累后人乎！他日或克就绪，愿与诸君落之。于是谋之，僚吏搜故府，得遗材千枚，取赢资以给其费。鸠工庀役，各举其任。嵗月讫，工民不与知像设礼器，百用具修。至于堂室。廊序。门牖。垣墙，皆一新之。'